In [ ]:
%pip install PyMuPDF requests

In [2]:
import sys
from dotenv import load_dotenv, find_dotenv

sys.path.append('../aisuite')

In [3]:
import os
def configure_environment(additional_env_vars=None):
    """
    Load environment variables from .env file and apply any additional variables.
    :param additional_env_vars: A dictionary of additional environment variables to apply.
    """
    # Load from .env file if available
    load_dotenv(find_dotenv())

    # Apply additional environment variables
    if additional_env_vars:
        for key, value in additional_env_vars.items():
            os.environ[key] = value

# Define additional API keys and credentials
additional_keys = {}

# Configure environment
configure_environment(additional_env_vars=additional_keys)

In [ ]:
import requests
import fitz
from io import BytesIO

# Link to paper in pdf format on the cost of avocados.
pdf_path = "https://arxiv.org/pdf/2104.04649"
pdf_text = ""
# Download PDF and load it into memory
response = requests.get(pdf_path)
if response.status_code == 200:
    pdf_data = BytesIO(response.content)  # Load PDF data into BytesIO
    # Open PDF from memory using fitz
    with fitz.open(stream=pdf_data, filetype="pdf") as pdf:
        text = ""
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            pdf_text += page.get_text("text")  # Extract text
            pdf_text += "\n" + "="*50 + "\n"  # Separator for each page
    print("Downloaded and extracted text from pdf.")
else:
    print(f"Failed to download PDF: {response.status_code}")

question = "Is the price of organic avocados higher than non-organic avocados? What has been the trend?"

In [9]:
import aisuite as ai
client = ai.Client()
messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer the question only based on the below text."},
    {"role": "user", "content": f"Answer the question based on the following text:\n\n{pdf_text}\n\nQuestion: {question}\nAnswer:"},
]

In [ ]:
anthropic_claude_3_opus = "anthropic:claude-3-5-sonnet-20240620"
response = client.chat.completions.create(model=anthropic_claude_3_opus, messages=messages)
print(response.choices[0].message.content)

In [ ]:
hf_model = "huggingface:mistralai/Mistral-7B-Instruct-v0.3"
response = client.chat.completions.create(model=hf_model, messages=messages)
print(response.choices[0].message.content)